# Kaggle Competition
### Maximovskaya Anastasiya

www.kaggle.com/c/dota-hse-ml

In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from pandas import get_dummies
from sklearn.model_selection import cross_val_score

In [8]:
df_train = pd.read_csv('main_train.csv')
df_test = pd.read_csv('main_test.csv')
tf_train = pd.read_csv('teamfights_train.csv')
tf_test = pd.read_csv('teamfights_test.csv')

In [3]:
df_train.head()

,match_id,radiant,hero,gold_0,lh_0,xp_0,gold_60,lh_60,xp_60,gold_120,...,xp_600,level_180,level_240,level_300,level_360,level_420,level_480,level_540,level_600,radiant_win
0,0,1,Rubick,0.0,0.0,0.0,100.0,0.0,46.0,250.0,...,1741.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0,1,Wraith King,0.0,0.0,0.0,175.0,2.0,124.0,526.0,...,2319.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
2,0,1,Riki,0.0,0.0,0.0,137.0,1.0,93.0,439.0,...,3859.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
3,0,1,Tusk,0.0,0.0,0.0,100.0,0.0,62.0,200.0,...,1676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,0,1,Templar Assassin,0.0,0.0,0.0,320.0,3.0,352.0,668.0,...,4453.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1


In [9]:
df_train, df_test = get_dummies(df_train).fillna(method='backfill'), get_dummies(df_test).fillna(method='backfill')

In [18]:
last_m_stat = ['level_600', 'gold_600', 'xp_600', 'lh_600']

In [22]:
cols = []
for i in df_train:
    if 'hero' in i:
        for j in last_m_stat:
            df_train[i+j] = df_train[i]*df_train[j]
            df_test[i+j] = df_test[i]*df_test[j]
            cols += [i+j]

In [ ]:
rad = df_train[df_train['radiant'] == 1].groupby('match_id')
dare = df_train[df_train['radiant'] == 0].groupby('match_id')
rs, ds = rad.sum()[cols], -dare.sum()[cols]

In [45]:
rs, ds = rs.reset_index(), ds.reset_index()

In [46]:
y_train = df_train[df_train['radiant'] == 1].groupby('match_id')['radiant_win'].max()
X_train = pd.merge(rs, ds, on=['match_id'])

In [48]:
rad = df_test[df_test['radiant'] == 1].groupby('match_id')
dare = df_test[df_test['radiant'] == 0].groupby('match_id')

rs, ds = rad.sum()[cols], -dare.sum()[cols]
rs, ds = rs.reset_index(), ds.reset_index()

In [49]:
X_test = pd.merge(rs, ds, on=['match_id'])

In [50]:
np.mean(cross_val_score(X=X_train, y=y_train, estimator=LogisticRegression(penalty='l2'), cv=3, scoring='roc_auc'))

0.74849184976709593

In [51]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
res = clf.predict_proba(X_test)
fl = pd.DataFrame()
fl['index'] = X_test['match_id']
fl['proba'] = res[:,1]
fl.to_csv('result', index=None)